In [ ]:
# Import dependencies
import scrapy

# Using scraping tool called 'spider' to go through the entire website
class MySpider(scrapy.Spider):
    name = 'myspider'
    start_urls = ['https://www.fragrantica.com/search/']

    def parse(self, response):
        # Extract data from the current page
        # For example:
        title = response.css('h1::text').get()
        yield {'title': title}

        # Follow links to other pages
        for next_page in response.css('a::attr(href)').getall():
            yield response.follow(next_page, callback=self.parse)